In [6]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [ ]:
"""
#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, 
with 6000 images per class. There are 50000 training images and 10000 test images. 

## data -- a 10000x3072(32*32*3) numpy array of uint8s. 
Each row of the array stores a 32x32 colour image. 
The first 1024 entries contain the red channel values, 
the next 1024 the green, and the final 1024 the blue. 
The image is stored in row-major order, so that the first 32 entries of the array 
are the red channel values of the first row of the image.

## labels -- a list of 10000 numbers in the range 0-9. 
The number at index i indicates the label of the ith image in the array data.
"""

In [7]:
"""
get data and its labels
"""
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        # we only need data & its labels 
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        """only shuffle for training data, not for testing data"""
        # read all data
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):  # zip: bound together,
                # binary classification, so we only need data with label say 0 & 1 
                if label in [0, 1]:
                    all_data.append(item)
                    all_labels.append(label)
                    
        self._data = np.vstack(all_data)  
        self._data = self._data / 127.5 - 1   # normalization, vital important !!
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0] # 10000 samples
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        """remove independency between data"""
        # eg: [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)  # p is one permutation
        self._data = self._data[p]  # apply this permutation to _data
        self._labels = self._labels[p]  # apply this permutation to _labels
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size  # end position
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 # reset indicator
                end_indicator = batch_size + 0
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        
        # get data from begin to end as one batch
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels


In [8]:
"""
test CifarData and its functions
"""
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

# train_data & test_data are two instances of class CifarData

train_data = CifarData(train_filenames, True) # we shuffle train data
test_data = CifarData(test_filenames, False) # we dont shuffle test data

tst_batch_data, tst_batch_labels = train_data.next_batch(20)
print(tst_batch_data)
print(tst_batch_labels)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)
[[ 0.69411765  0.65490196  0.67843137 ..., -0.5372549  -0.52156863
  -0.52941176]
 [-0.77254902 -0.99215686 -0.97647059 ..., -0.09803922 -0.11372549
  -0.00392157]
 [ 0.49803922  0.43529412  0.39607843 ..., -0.35686275 -0.35686275
  -0.30980392]
 ..., 
 [ 0.70196078  0.65490196  0.75686275 ..., -0.74901961 -0.73333333
  -0.70980392]
 [ 0.69411765  0.65490196  0.42745098 ..., -0.27843137 -0.34117647
  -0.35686275]
 [ 0.78039216  0.79607843  0.79607843 ...,  0.43529412  0.41176471
   0.39607843]]
[0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0]


In [9]:
"""
 create the compute graph
"""
# [None, 3072]
x = tf.placeholder(tf.float32, [None, 3072]) # [number of samples, length of sample]
# [None]
y = tf.placeholder(tf.int64, [None])

# binary classification need one output
# (3072, 1)
w = tf.get_variable('w', [x.get_shape()[-1], 1],  #[-1]: get 3072 from x. 1: one output
                   initializer=tf.random_normal_initializer(0, 1))
# (1, )
b = tf.get_variable('b', [1],
                   initializer=tf.constant_initializer(0.0))

# [None, 3072] * [3072, 1] = [None, 1]
y_ = tf.matmul(x, w) + b

# turn y_ to probability value
p_y_1 = tf.nn.sigmoid(y_) # [None, 1]

# reshape y from [None] to [None, 1]
y_reshaped = tf.reshape(y, (-1, 1)) # [None, 1]

# change type
y_reshaped_float = tf.cast(y_reshaped, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

# when p_y_1 > 0,5, then set predict as true
predict = p_y_1 > 0.5  # bool

# correct_prediction possible value: [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# AdamOptimizer is one kind of GD
with tf.name_scope('train_op'):
    # apply GD to loss
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [11]:
"""
execute the graph
"""
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run( 
                                        [loss, accuracy, train_op], 
                                        feed_dict={
                                            x: batch_data,
                                            y: batch_labels}
                                        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        
        # testing while trainning
        # after training, all w and b were changed, so sess.run will run a trained model
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                                        [accuracy],
                                        feed_dict = {
                                            x: test_batch_data, 
                                            y: test_batch_labels}
                                        )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)  # get mean value
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))
                
                
            

[Train] Step: 500, loss: 0.50180, acc: 0.45000
[Train] Step: 1000, loss: 0.20000, acc: 0.80000
[Train] Step: 1500, loss: 0.17230, acc: 0.80000
[Train] Step: 2000, loss: 0.36599, acc: 0.60000
[Train] Step: 2500, loss: 0.14963, acc: 0.85000
[Train] Step: 3000, loss: 0.15000, acc: 0.85000
[Train] Step: 3500, loss: 0.34926, acc: 0.60000
[Train] Step: 4000, loss: 0.17098, acc: 0.80000
[Train] Step: 4500, loss: 0.05039, acc: 0.95000
[Train] Step: 5000, loss: 0.10699, acc: 0.90000
(2000, 3072)
(2000,)
[Test ] Step: 5000, acc: 0.80100
[Train] Step: 5500, loss: 0.25000, acc: 0.75000
[Train] Step: 6000, loss: 0.24986, acc: 0.75000
[Train] Step: 6500, loss: 0.13770, acc: 0.85000
[Train] Step: 7000, loss: 0.28002, acc: 0.70000
[Train] Step: 7500, loss: 0.20000, acc: 0.80000
[Train] Step: 8000, loss: 0.15190, acc: 0.85000
[Train] Step: 8500, loss: 0.05004, acc: 0.95000
[Train] Step: 9000, loss: 0.25435, acc: 0.75000
[Train] Step: 9500, loss: 0.29999, acc: 0.70000
[Train] Step: 10000, loss: 0.15409,

[Train] Step: 76000, loss: 0.13292, acc: 0.85000
[Train] Step: 76500, loss: 0.20359, acc: 0.80000
[Train] Step: 77000, loss: 0.05104, acc: 0.95000
[Train] Step: 77500, loss: 0.05009, acc: 0.95000
[Train] Step: 78000, loss: 0.00005, acc: 1.00000
[Train] Step: 78500, loss: 0.15000, acc: 0.85000
[Train] Step: 79000, loss: 0.15778, acc: 0.85000
[Train] Step: 79500, loss: 0.15000, acc: 0.85000
[Train] Step: 80000, loss: 0.00044, acc: 1.00000
(2000, 3072)
(2000,)
[Test ] Step: 80000, acc: 0.81900
[Train] Step: 80500, loss: 0.05003, acc: 0.95000
[Train] Step: 81000, loss: 0.10018, acc: 0.90000
[Train] Step: 81500, loss: 0.12571, acc: 0.85000
[Train] Step: 82000, loss: 0.15003, acc: 0.85000
[Train] Step: 82500, loss: 0.15003, acc: 0.85000
[Train] Step: 83000, loss: 0.10008, acc: 0.90000
[Train] Step: 83500, loss: 0.25010, acc: 0.75000
[Train] Step: 84000, loss: 0.19999, acc: 0.80000
[Train] Step: 84500, loss: 0.10318, acc: 0.90000
[Train] Step: 85000, loss: 0.20302, acc: 0.80000
(2000, 3072)
(